In [1]:
import pandas as pd 
import wandb
import re
api = wandb.Api()
r = []
def read(r):
    for run in runs: 
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

        metadata_list.append(run.metadata)

        hist_list.append(run.history())

        system_metrics = run.history(stream="events")
        allocated = sorted([x for x in system_metrics.columns if re.match('system\.gpu\..\.memoryAllocated$', x)])
        allocated_bytes = sorted([x for x in system_metrics.columns if re.match('system\.gpu\..\.memoryAllocatedBytes$', x)])
        system_metrics['allocatedgb'] = round(system_metrics[allocated_bytes].sum(axis=1)*1e-09,2)
        system_metrics['allocated'] = round(system_metrics[allocated].mean(axis=1))
        gpus = {'allocatedmax':system_metrics['allocated'].max(),
        'allocatedmed':system_metrics['allocated'].median(),
        'allocatedgbmax':system_metrics['allocatedgb'].max(),
        'allocatedgbmed':system_metrics['allocatedgb'].median()}
        sys_list.append(gpus)

# Project is specified by <entity/project-name>
summary_list, config_list, name_list, metadata_list, hist_list, sys_list = [], [], [], [],[],[]
runs = api.runs("openfact/clef23_p4_llama")
read(runs)
runs = api.runs("checkw/clef23_p5_llama_floki")
read(runs)


In [2]:
runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list,
    "metadata": metadata_list,
    "sys":sys_list,
    })

runs_df.to_csv("project.csv")

In [3]:
dfs = [runs_df[['name']],
        pd.DataFrame(runs_df.name.str.split('|').to_list(), columns=['model','o','dataset','rand']),
        pd.json_normalize(runs_df.metadata),
        pd.json_normalize(runs_df.config),
        pd.json_normalize(runs_df.summary),
        pd.json_normalize(runs_df.sys),
]
data = pd.concat(dfs, axis=1)
data.head(2)



,name,model,o,dataset,rand,os,python,heartbeatAt,startedAt,docker,...,train/train_runtime,eval/samples_per_second,train/train_samples_per_second,train/total_flos,train/learning_rate,_wandb.runtime,allocatedmax,allocatedmed,allocatedgbmax,allocatedgbmed
0,llama13|adamw_torch 8 1e-05 f1_bi 2|r21|0,llama13,adamw_torch 8 1e-05 f1_bi 2,r21,0,Linux-5.10.0-21-amd64-x86_64-with-glibc2.29,3.8.10,2023-11-16T01:47:01.388901,2023-11-16T01:47:00.633252,None,...,22680.5925,6.397,0.678,1.312225e+17,0.000004,23040.0,41.0,40.0,19.14,18.89
1,llama7|paged_adamw_8bit 8 1e-05 f1_bi 2|r21|0,llama7,paged_adamw_8bit 8 1e-05 f1_bi 2,r21,0,Linux-5.10.0-21-amd64-x86_64-with-glibc2.29,3.8.10,2023-11-15T20:38:35.695311,2023-11-15T20:38:34.820752,None,...,18209.3552,11.445,0.845,9.511740e+16,0.000002,18418.0,27.0,26.0,12.55,12.40


In [4]:
data['hw'] = data['gpu']+" x "+data['gpu_count'].astype(str)
data['s'] = round(data['_runtime']/60)
data['time(h)']=round(data['_runtime']/60/60,2)
cols = ['name','model','host','dataset','codePath','hw','model_type','metric_for_best_model','optim','s',
'_runtime','time(h)',
'allocatedmax','allocatedmed','allocatedgbmax','allocatedgbmed',
'train/train_runtime','eval/runtime',
'train/global_step',
'train/total_flos',
'train/train_steps_per_second','train/train_samples_per_second',
'eval/steps_per_second','eval/samples_per_second',
'train/train_loss','train/loss','eval/loss',
'eval/f1_bi','eval/f1_ma','eval/prec','eval/rec','eval/acc',
'test_f1','test_precision','test_recall','test_accuracy',
'dev_test_f1','dev_test_precision','dev_test_recall','dev_test_accuracy'
]
datat = data[cols].T
datat

,0,1,2,3,4,5,6
name,llama13|adamw_torch 8 1e-05 f1_bi 2|r21|0,llama7|paged_adamw_8bit 8 1e-05 f1_bi 2|r21|0,llama70|paged_adamw_8bit 8 1e-05 f1_bi 2|r21|0,llama7|paged_adamw_8bit 8 1e-05 f1_bi 2|r21|0,llama13|paged_adamw_8bit 8 1e-05 f1_bi 2|r21|0,llama7|adamw_torch 8 1e-05 f1_bi 2|r21|0,llama7|adamw_torch 8 1e-05 f1_bi 2|r21|0
model,llama13,llama7,llama70,llama7,llama13,llama7,llama7
host,c301b88b57d6,c301b88b57d6,clef-l4-8,clef-l4-4,c301b88b57d6,clef-l4-4,c301b88b57d6
dataset,r21,r21,r21,r21,r21,r21,r21
codePath,cp_train_template_llama.py,cp_train_template_llama.py,cp_train_template_llama.py,cp_train_template_llama.py,cp_train_template_llama.py,cp_train_template_llama.py,cp_train_template_llama.py
hw,NVIDIA GeForce RTX 2080 Ti x 4,NVIDIA GeForce RTX 2080 Ti x 4,NVIDIA L4 x 8,NVIDIA L4 x 4,NVIDIA GeForce RTX 2080 Ti x 4,NVIDIA L4 x 4,NVIDIA GeForce RTX 2080 Ti x 4
model_type,llama,llama,llama,llama,llama,llama,llama
metric_for_best_model,f1_bi,f1_bi,f1_bi,f1_bi,f1_bi,f1_bi,f1_bi
optim,adamw_torch,paged_adamw_8bit,paged_adamw_8bit,paged_adamw_8bit,paged_adamw_8bit,adamw_torch,adamw_torch
s,384.0,307.0,1515.0,470.0,394.0,278.0,217.0


In [7]:
piv = data.pivot_table(index=['dataset','hw', 'optim'], columns=['model'], values=['test_f1','eval/f1_bi','allocatedgbmax','allocatedmax','time(h)'], aggfunc=['mean'],fill_value='-')
piv.round()

mean         \
                                                        allocatedgbmax          
model                                                          llama13 llama7   
dataset hw                             optim                                    
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch               19.14  12.31   
                                       paged_adamw_8bit          18.14  12.55   
        NVIDIA L4 x 4                  adamw_torch                   -   16.7   
                                       paged_adamw_8bit              -  16.92   
        NVIDIA L4 x 8                  paged_adamw_8bit              -      -   

                                                                              \
                                                                allocatedmax   
model                                                   llama70      llama13   
dataset hw                             optim                                   
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch            -         41.0   
                                       paged_adamw_8bit       -         38.0   
        NVIDIA L4 x 4                  adamw_torch            -            -   
                                       paged_adamw_8bit       -            -   
        NVIDIA L4 x 8                  paged_adamw_8bit    85.0            -   

                                                                        \
                                                                         
model                                                   llama7 llama70   
dataset hw                             optim                             
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch        26.0       -   
                                       paged_adamw_8bit   27.0       -   
        NVIDIA L4 x 4                  adamw_torch        17.0       -   
                                       paged_adamw_8bit   18.0       -   
        NVIDIA L4 x 8                  paged_adamw_8bit      -    44.0   

                                                                              \
                                                        eval/f1_bi             
model                                                      llama13    llama7   
dataset hw                             optim                                   
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch        0.865225  0.863866   
                                       paged_adamw_8bit   0.865672  0.860927   
        NVIDIA L4 x 4                  adamw_torch               -  0.861953   
                                       paged_adamw_8bit          -  0.868167   
        NVIDIA L4 x 8                  paged_adamw_8bit          -         -   

                                                                           \
                                                                  test_f1   
model                                                     llama70 llama13   
dataset hw                             optim                                
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch              -   0.814   
                                       paged_adamw_8bit         -   0.821   
        NVIDIA L4 x 4                  adamw_torch              -       -   
                                       paged_adamw_8bit         -       -   
        NVIDIA L4 x 8                  paged_adamw_8bit  0.843077       -   

                                                                        \
                                                                         
model                                                   llama7 llama70   
dataset hw                             optim                             
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch       0.812       -   
                                       paged_adamw_8bit  0.834       -   
        NVIDIA L4 x 4                  adamw_torch           -       -   
               

In [6]:
data.pivot_table(index=['dataset','hw', 'optim'], columns=['model'], values=['test_f1','eval/f1_bi','allocatedgbmax','time(h)'], aggfunc=['count'],fill_value='-')

count         \
                                                        allocatedgbmax          
model                                                          llama13 llama7   
dataset hw                             optim                                    
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch                   1      1   
                                       paged_adamw_8bit              1      1   
        NVIDIA L4 x 4                  adamw_torch                   -      1   
                                       paged_adamw_8bit              -      1   
        NVIDIA L4 x 8                  paged_adamw_8bit              -      -   

                                                                            \
                                                                eval/f1_bi   
model                                                   llama70    llama13   
dataset hw                             optim                                 
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch            -          1   
                                       paged_adamw_8bit       -          1   
        NVIDIA L4 x 4                  adamw_torch            -          -   
                                       paged_adamw_8bit       -          -   
        NVIDIA L4 x 8                  paged_adamw_8bit       1          -   

                                                                        \
                                                                         
model                                                   llama7 llama70   
dataset hw                             optim                             
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch           1       -   
                                       paged_adamw_8bit      1       -   
        NVIDIA L4 x 4                  adamw_torch           1       -   
                                       paged_adamw_8bit      1       -   
        NVIDIA L4 x 8                  paged_adamw_8bit      -       1   

                                                                        \
                                                        test_f1          
model                                                   llama13 llama7   
dataset hw                             optim                             
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch            1      1   
                                       paged_adamw_8bit       1      1   
        NVIDIA L4 x 4                  adamw_torch            -      0   
                                       paged_adamw_8bit       -      1   
        NVIDIA L4 x 8                  paged_adamw_8bit       -      -   

                                                                         \
                                                                time(h)   
model                                                   llama70 llama13   
dataset hw                             optim                              
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch            -       1   
                                       paged_adamw_8bit       -       1   
        NVIDIA L4 x 4                  adamw_torch            -       -   
                                       paged_adamw_8bit       -       -   
        NVIDIA L4 x 8                  paged_adamw_8bit       0       -   

                                                                        
                                                                        
model                                                   llama7 llama70  
dataset hw                             optim                            
r21     NVIDIA GeForce RTX 2080 Ti x 4 adamw_torch           1       -  
                                       paged_adamw_8bit      1       -  
        NVIDIA L4 x 4                  adamw_torch           1       -  
                                       paged_adamw_8bit      1       -  
        NVIDIA L4 x 8                 